In [20]:
import pandas as pd
import numpy as np
import torch
import pyro
import pyro.distributions as dist
import pyro.infer
import pyro.optim
from pyro.nn import PyroSample, PyroModule
from pyro.infer.autoguide import AutoDiagonalNormal, AutoLowRankMultivariateNormal
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os





In [21]:
df = pd.read_csv(r'/Users/jamesjirsa/Desktop/Data_Science/JJ_Projects/UFC/new_method/scrape_ufc_stats/final_fight_data.csv')
df

,Unnamed: 0,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,...,delta_ELO_FIGHTER,delta_AGE,delta_REACH,delta_HEIGHT,delta_SIG_STR_vs_avoid,delta_TD_vs_avoid,PC1,PC2,opponent_PC1,opponent_PC2
0,0,UFC25:UltimateJapan3,TitoOrtizvs.WanderleiSilva,W/L,UFC Light Heavyweight Title Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),John McCarthy,...,-45.805984,-1.442847,0.0,-4.0,5.852806,-0.025210,-2309.770770,111.777029,123.244200,65.610081
1,1,UFC25:UltimateJapan3,TitoOrtizvs.WanderleiSilva,W/L,UFC Light Heavyweight Title Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),John McCarthy,...,45.805984,1.442847,0.0,4.0,0.812415,-0.099214,123.244200,65.610081,-2309.770770,111.777029
2,4,UFC27:UltimateBadBoyz,JeremyHornvs.EugeneJackson,W/L,Middleweight Bout,Submission,1,4:32,3 Rnd (5-5-5),Mario Yamasaki,...,22.284177,-8.919918,NaN,5.0,-0.581752,0.029975,-846.219711,-360.531587,NaN,NaN
3,5,UFC28:HighStakes,RandyCouturevs.KevinRandleman,W/L,UFC Heavyweight Title Bout,KO/TKO,3,4:13,5 Rnd (5-5-5-5-5),John McCarthy,...,44.264592,8.134155,NaN,3.0,1.683676,0.236066,-1103.921141,-426.244419,NaN,NaN
4,6,UFC28:HighStakes,JensPulvervs.JohnLewis,W/L,Lightweight Bout,KO/TKO,1,0:15,3 Rnd (5-5-5),Mario Yamasaki,...,27.873931,NaN,NaN,-5.0,0.894140,-0.006089,-310.385372,125.252047,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10398,11693,UFCFightNight:Sandhagenvs.Nurmagomedov,ViktoriiaDudakovavs.SamHughes,L/W,Women's Strawweight Bout,Decision - Split,3,5:00,3 Rnd (5-5-5),Daniel Movahedi,...,-53.379668,6.579055,NaN,NaN,-3.026368,0.064299,4525.143067,-1018.395761,NaN,NaN
10399,11694,UFCFightNight:Sandhagenvs.Nurmagomedov,SharaMagomedovvs.MichalOleksiejczuk,W/L,Middleweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Daniel Movahedi,...,-24.699481,-0.772074,NaN,-2.0,3.331041,-0.523622,2107.863170,-1553.580021,NaN,NaN
10400,11695,UFCFightNight:Sandhagenvs.Nurmagomedov,TonyFergusonvs.MichaelChiesa,L/W,Welterweight Bout,Submission,1,3:44,3 Rnd (5-5-5),Marc Goddard,...,44.625080,-3.816564,-1.0,2.0,-3.546148,0.076678,8868.919814,-158.645274,13129.879716,-6580.190969
10401,11696,UFCFightNight:Sandhagenvs.Nurmagomedov,MarlonVeravs.DeivesonFigueiredo,L/W,Bantamweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Keith Peterson,...,-57.771347,-4.958248,2.0,3.0,0.794614,-0.137185,15885.407700,-6412.125198,9011.455859,-2906.188676


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10403 entries, 0 to 10402
Columns: 177 entries, Unnamed: 0 to opponent_PC2
dtypes: float64(159), int64(4), object(14)
memory usage: 14.0+ MB


In [23]:
max_fight_idx = df.groupby('FIGHTER')['Fight_Number'].idxmax()
max_fight_rows = df.loc[max_fight_idx]
extra = max_fight_rows[['FIGHTER', 'PC1', 'PC2']]
extra = extra.reset_index(drop=True)
extra

,FIGHTER,PC1,PC2
0,AJDobson,203.796394,-684.489701
1,AJFletcher,-186.817878,-240.184326
2,AaronPhillips,-776.704717,-475.177970
3,AaronRiley,2578.040225,-922.712749
4,AaronRosa,-912.749506,-419.983606
...,...,...,...
1539,ZhalgasZhumagulov,572.936588,-812.903884
1540,ZhangLipeng,360.553828,-117.202185
1541,ZhangTiequan,-2470.537479,181.515467
1542,ZhangWeili,5546.337955,-1268.547434


In [24]:
df = df.drop(['PC1', 'PC2'], axis = 1)

In [25]:
df['RESULT'] = np.where(df['RESULT'] == 'Win', 1,0)

In [26]:
features = df.columns[df.columns.get_loc('ELO_FIGHTER'):].tolist()
#features.remove('CUM_index')
features.remove('DOB')
df = df.dropna(subset = 'opponent_PC1')
#df = df.drop(['CUM_index'], axis = 1)

In [27]:
abc = df[(df['DATE'] < '2005')]
abc = abc[features]


X_train = df[(df['DATE'] >= '2005') & (df['DATE'] < '2023')]
y_train = X_train['RESULT']
print(X_train.DATE.max())
X_train = X_train[features]

X_test = df[(df['DATE'] >= '2023') & (df['DATE'] < '2024')]
y_test = X_test['RESULT']
X_test = X_test[features]

X_val = df[(df['DATE'] >= '2024')]
y_val = X_val['RESULT']
X_val = X_val[features]


2022-12-17


In [28]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)
abc_scaled = scaler.transform(abc)

In [29]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns = X_test.columns)
X_val_scaled = pd.DataFrame(X_val_scaled, columns = X_val.columns)
abc_scaled  = pd.DataFrame(abc_scaled, columns = abc.columns)

In [30]:
X_train.shape

(9020, 158)

In [31]:
X_train_scaled

,ELO_FIGHTER,CUM_KD,CUM_SIG.STR._Landed,CUM_SIG.STR._Attempted,CUM_SIG.STR._Attempted_avoided,CUM_TOTAL STR._Landed,CUM_TOTAL STR._Attempted,CUM_TOTAL STR._Attempted_avoided,CUM_TD_Landed,CUM_TD_Attempted,...,delta_CUM_GROUND_Landed_PM,delta_CUM_SUB.ATT_PM,delta_ELO_FIGHTER,delta_AGE,delta_REACH,delta_HEIGHT,delta_SIG_STR_vs_avoid,delta_TD_vs_avoid,opponent_PC1,opponent_PC2
0,0.248139,0.769543,-0.504221,-0.546830,-0.565189,-0.638842,-0.631079,-0.591191,-0.685737,-0.692386,...,-0.414060,0.446222,0.144171,-0.563464,-0.913979,-1.995668,-0.159784,-0.729390,-0.901345,0.638326
1,-0.719262,-0.369616,-0.659646,-0.629953,-0.713998,-0.552265,-0.586917,-0.724418,-0.504929,-0.580706,...,-0.155445,-0.125488,-0.462714,1.549790,-1.218638,0.399134,-0.131731,0.105043,-0.591187,0.557743
2,-0.318515,-0.749336,-0.830613,-0.794502,-0.616817,-0.798468,-0.794907,-0.617837,0.308707,0.014924,...,0.155445,0.125488,0.462714,-1.549790,1.218638,-0.399134,-0.063439,1.923466,-0.554601,0.266547
3,0.123275,0.010103,-0.360453,-0.506117,-0.762589,-0.549559,-0.604012,-0.783630,-0.685737,-0.729613,...,0.414060,-0.446222,-0.144171,0.563464,0.913979,1.995668,3.597300,-1.220989,-0.638712,0.156029
4,1.303808,-0.369616,0.560441,0.231811,-0.197720,1.955767,1.045662,-0.173746,2.659211,1.392317,...,0.543198,-0.639070,0.630973,0.422334,0.000000,1.197401,0.534409,1.193470,-0.452746,-0.066829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9015,0.846508,-0.369616,-0.197256,-0.226213,-0.422453,-0.279006,-0.253563,-0.431319,-0.595333,-0.059530,...,-0.195186,-0.858231,0.873302,0.696665,0.304660,0.000000,1.297544,-0.832062,-0.619763,0.822012
9016,0.090158,-0.749336,-0.725701,-0.767360,-0.832439,-0.425104,-0.575520,-0.783630,0.308707,0.089377,...,0.195186,0.858231,-0.873302,-0.696665,-0.304660,0.000000,-0.222487,1.597109,-0.412399,-0.114677
9017,-1.059828,-0.369616,-0.593590,-0.501028,-0.486229,-0.738947,-0.609710,-0.511255,-0.595333,-0.617933,...,-0.090685,0.209981,-0.428331,0.052329,0.000000,-0.399134,-0.384122,-0.578205,-0.366346,0.513558
9018,1.824815,0.769543,3.525174,3.838325,3.889988,2.502285,3.265172,3.814184,0.670323,0.201058,...,0.028789,0.142765,0.777682,-1.342059,-0.304660,0.798267,0.226601,-0.526152,1.079822,-2.159894


In [32]:
import torch
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
import torch.nn.functional as F
import torch.nn as nn

pyro.clear_param_store()

class SimpleBayesianNN(PyroModule):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        
        self.hidden1 = PyroModule[torch.nn.Linear](input_size, hidden_size)
        self.hidden1.weight = PyroSample(dist.Uniform(-1., 1.).expand([hidden_size, input_size]).to_event(2))
        self.hidden1.bias = PyroSample(dist.Uniform(-0.01, .01).expand([hidden_size]).to_event(1))

        self.output = PyroModule[torch.nn.Linear](hidden_size, output_size)
        self.output.weight = PyroSample(dist.Uniform(-1., 1.).expand([output_size, hidden_size]).to_event(2))
        self.output.bias = PyroSample(dist.Uniform(-0.01, .01).expand([output_size]).to_event(1))

    def forward(self, x):
        hidden1 = F.leaky_relu(self.hidden1(x))
        hidden1 = F.dropout(hidden1, p=0.2, training=self.training)  
        output = self.output(hidden1)
        return output
    
bnn = SimpleBayesianNN(input_size=X_train.shape[1], hidden_size=200, output_size=1)

def model(x_data, y_data=None):
    logits = bnn(x_data).squeeze(-1)
    probabilities = torch.sigmoid(logits)  
    with pyro.plate("data", x_data.shape[0]):
        pyro.sample("obs", dist.Bernoulli(probs=probabilities), obs=y_data)
    return logits 

#Changed this
guide = pyro.infer.autoguide.AutoNormal(model)  


x_tensor = torch.tensor(X_train_scaled.values, dtype=torch.float)
y_tensor = torch.tensor(y_train.values, dtype=torch.float)

guide(x_tensor, y_tensor)

x_tensor_test = torch.tensor(X_test_scaled.values, dtype=torch.float)
y_tensor_test = torch.tensor(y_test.values, dtype=torch.float)


x_tensor_val = torch.tensor(X_val_scaled.values, dtype=torch.float)
y_tensor_val = torch.tensor(y_val.values, dtype=torch.float)


In [33]:
class EarlyStopper:
    def __init__(self, patience=500, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False
    

def save_checkpoint(model, pyro_optimizer, scheduler, epoch, loss, path="checkpoint.pth"):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': pyro_optimizer.get_state(),  
        'scheduler_state_dict': scheduler.scheduler.state_dict(),  
        'loss': loss
    }, path)
    print(f"Checkpoint saved at epoch {epoch}.")

def load_checkpoint(model, pyro_optimizer, scheduler, path="checkpoint.pth"):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    pyro_optimizer.set_state(checkpoint['optimizer_state_dict'])  
    scheduler.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])  
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    print(f"Checkpoint loaded. Resuming from epoch {epoch} with loss {loss:.4f}")
    return epoch, loss

def load_model_only(model, path="checkpoint.pth"):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])


def save_best_model(model, val_loss, best_val_loss, model_save_path="best_model.pth"):
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), model_save_path)
        print(f"Best model saved with val_loss: {val_loss:.4f}")
    return best_val_loss

def load_best_model(model, model_save_path="best_model.pth"):
    model.load_state_dict(torch.load(model_save_path))
    model.eval()  # Set to evaluation mode
    print("Best model loaded.")


early_stopper = EarlyStopper(patience=2000, min_delta=0.01)

In [34]:
from pyro.optim import PyroOptim
from pyro.optim.lr_scheduler import PyroLRScheduler
import torch.optim as optim

optimizer = PyroOptim(optim.Adam, {"lr": 1e-3, "weight_decay": 0.01})
scheduler = pyro.optim.ReduceLROnPlateau({
    'optimizer': optimizer,
    'optim_args': {'lr': 0.01},  
    'factor': 1,               
    'patience': 50,              
    'verbose': True,             
    'mode': 'min',               
    'gamma': 0.1                 
})


svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

def focal_loss(logits, targets, alpha=0.25, gamma=2):
    BCE_loss = F.binary_cross_entropy_with_logits(logits, targets, reduction='none')
    pt = torch.exp(-BCE_loss)
    F_loss = alpha * (1-pt)**gamma * BCE_loss
    return F_loss.mean()

def compute_validation_loss(x_test, y_test):
    with torch.no_grad():
        logits = bnn(x_test).squeeze(-1)
        val_loss = focal_loss(logits, y_test)
    return val_loss.item()

def compute_validation_loss_no_foc(x_test, y_test):
    with torch.no_grad():
        logits = bnn(x_test).squeeze(-1)
        val_loss = F.binary_cross_entropy_with_logits(logits, y_test)
    return val_loss.item()


best_val_loss = float('inf')
num_epochs = 5000
checkpoint_path = 'best_model.pth'

for epoch in range(num_epochs):
    train_loss = svi.step(x_tensor, y_tensor)
    val_loss = compute_validation_loss_no_foc(x_tensor_test, y_tensor_test)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(bnn.state_dict(), "best_model.pth")  
        print(f"Model saved at epoch {epoch} with validation loss {val_loss}")


    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}, Best is = {best_val_loss:.4f}")

bnn.load_state_dict(torch.load("best_model.pth"))
bnn.eval()


predictive = pyro.infer.Predictive(model, guide=guide, num_samples=5000, return_sites=["_RETURN", 'obs'])
predictions = predictive(x_tensor, None)
logits_samples = predictions["_RETURN"]
probability_samples = torch.sigmoid(logits_samples)
mean_probabilities = torch.mean(probability_samples, dim=0)
predicted_labels = (mean_probabilities > 0.5).float()
accuracy = (predicted_labels == y_tensor).float().mean()
accuracy


Model saved at epoch 0 with validation loss 14.79515266418457
Epoch 0: Train Loss = 78940.4945, Val Loss = 14.7952, Best is = 14.7952
Model saved at epoch 11 with validation loss 13.95357894897461
Model saved at epoch 41 with validation loss 13.231770515441895
Model saved at epoch 58 with validation loss 12.323918342590332
Epoch 100: Train Loss = 76089.9943, Val Loss = 16.0931, Best is = 12.3239
Epoch 200: Train Loss = 73383.0861, Val Loss = 18.8265, Best is = 12.3239
Model saved at epoch 241 with validation loss 11.00731372833252
Epoch 300: Train Loss = 70549.2490, Val Loss = 16.5540, Best is = 11.0073
Epoch 400: Train Loss = 67611.2188, Val Loss = 19.8365, Best is = 11.0073
Epoch 500: Train Loss = 64988.1507, Val Loss = 19.3649, Best is = 11.0073
Epoch 600: Train Loss = 62530.2821, Val Loss = 16.4596, Best is = 11.0073
Epoch 700: Train Loss = 59950.6802, Val Loss = 16.2135, Best is = 11.0073
Epoch 800: Train Loss = 58022.4306, Val Loss = 33.7177, Best is = 11.0073
Epoch 900: Train Lo

tensor(0.6115)

In [35]:
lower_bound = torch.quantile(probability_samples, 0.025, dim=0)  
upper_bound = torch.quantile(probability_samples, 0.975, dim=0)
std = torch.std(probability_samples, dim=0)
skew = torch.mean(((probability_samples - mean_probabilities) ** 3), dim=0) / (std ** 3)
kurtosis = torch.mean(((probability_samples - mean_probabilities) ** 4), dim=0) / (std ** 4)


In [36]:
data = np.column_stack((mean_probabilities.numpy(), lower_bound.numpy(), upper_bound.numpy(), std.numpy(), skew.numpy(), kurtosis.numpy()))
datafr = pd.DataFrame(data, columns = ['proba', 'lower', 'upper', 'std', 'skew', 'kurtosis'])
datafr

,proba,lower,upper,std,skew,kurtosis
0,0.515769,0.136058,0.875587,0.200699,-0.061243,2.188041
1,0.364336,0.109357,0.690572,0.154319,0.383651,2.632828
2,0.674047,0.279855,0.945108,0.177630,-0.545604,2.665932
3,0.559568,0.067592,0.966330,0.269550,-0.228596,1.886160
4,0.575966,0.123851,0.943693,0.229840,-0.256142,2.127746
...,...,...,...,...,...,...
9015,0.517323,0.223622,0.802645,0.153244,-0.064796,2.449592
9016,0.635521,0.118338,0.975773,0.248892,-0.529354,2.253780
9017,0.455324,0.178167,0.753966,0.150832,0.118460,2.461317
9018,0.578122,0.072399,0.969919,0.268804,-0.297308,1.940047


In [37]:
predictions = predictive(x_tensor_val, None)

In [38]:
logits_samples = predictions["_RETURN"]
probability_samples = torch.sigmoid(logits_samples)
mean_probabilities = torch.mean(probability_samples, dim=0)
predicted_labels = (mean_probabilities > 0.5).float()

accuracy = (predicted_labels == y_tensor_val).float().mean()
accuracy

tensor(0.6250)

In [39]:
lower_bound = torch.quantile(probability_samples, 0.025, dim=0)  
upper_bound = torch.quantile(probability_samples, 0.975, dim=0)
std = torch.std(probability_samples, dim=0)
skew = torch.mean(((probability_samples - mean_probabilities) ** 3), dim=0) / (std ** 3)
kurtosis = torch.mean(((probability_samples - mean_probabilities) ** 4), dim=0) / (std ** 4)
median = torch.median(probability_samples, dim=0)


datafr = {
    'proba': mean_probabilities.numpy(),
    'median': median.values.numpy(),
    'lower': lower_bound.numpy(),
    'upper': upper_bound.numpy(),
    'std': std.numpy(),
    'skew': skew.numpy(),
    'kurtosis': kurtosis.numpy(),
    'trace': [probability_samples[i] for i in range(mean_probabilities.shape[0])]  # Store each trace as an array in the 'trace' column
}

datafr = pd.DataFrame(datafr)

datafr = datafr.reset_index(drop =  True)

extra_val = df[(df['DATE'] >= '2024')]
extra_val = extra_val.reset_index(drop = True)

xtra_feats =['EVENT',
 'BOUT',
 'OUTCOME',
 'WEIGHTCLASS',
 'METHOD',
 'ROUND',
 'TIME',
 'TIME FORMAT',
 'REFEREE',
 'DETAILS',
 'URL',
 'FIGHTER',
 'RESULT',
 'TIME_IN_SECONDS',
 'DATE',
 'ELO_FIGHTER',]

extra_val = pd.concat([extra_val[xtra_feats], datafr], axis = 1)
extra_val.to_csv(r'ProbaPlus.csv')
extra_val['Estimate'] = np.where(extra_val['proba'] > .5,1,0)

In [40]:
extra_val

,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,...,ELO_FIGHTER,proba,median,lower,upper,std,skew,kurtosis,trace,Estimate
0,UFCFightNight:Ankalaevvs.Walker2,MagomedAnkalaevvs.JohnnyWalker,W/L,Light Heavyweight Bout,KO/TKO,2,2:42,5 Rnd (5-5-5-5-5),Marc Goddard,Punch to Head At Distance,...,1726.390024,0.580108,0.586625,0.288613,0.838573,0.144345,-0.222904,2.601757,"[tensor(0.7603), tensor(0.5474), tensor(0.9006...",1
1,UFCFightNight:Ankalaevvs.Walker2,JimMillervs.GabrielBenitez,W/L,Lightweight Bout,Submission,3,3:25,3 Rnd (5-5-5),Dan Miragliotta,Neck Crank From Back Control,...,1509.527752,0.458526,0.451277,0.114017,0.839509,0.195997,0.127930,2.261112,"[tensor(0.5753), tensor(0.4763), tensor(0.0156...",0
2,UFCFightNight:Ankalaevvs.Walker2,JimMillervs.GabrielBenitez,W/L,Lightweight Bout,Submission,3,3:25,3 Rnd (5-5-5),Dan Miragliotta,Neck Crank From Back Control,...,1606.672231,0.432317,0.387055,0.012441,0.969077,0.306722,0.280852,1.755708,"[tensor(0.5394), tensor(0.2533), tensor(0.0196...",0
3,UFCFightNight:Ankalaevvs.Walker2,MagomedAnkalaevvs.JohnnyWalker,W/L,Light Heavyweight Bout,KO/TKO,2,2:42,5 Rnd (5-5-5-5-5),Marc Goddard,Punch to Head At Distance,...,1622.106542,0.509351,0.509032,0.139376,0.874310,0.199340,-0.019269,2.198014,"[tensor(0.6817), tensor(0.5073), tensor(0.0386...",1
4,UFCFightNight:Ankalaevvs.Walker2,MatthewSemelsbergervs.PrestonParsons,L/W,Welterweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Dan Miragliotta,Mike Bell 27 - 30.Junichiro Kamijo 27 - 30.Ron...,...,1482.314337,0.547292,0.550161,0.247168,0.827564,0.152949,-0.127112,2.495623,"[tensor(0.7042), tensor(0.7541), tensor(0.8722...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,UFCFightNight:Sandhagenvs.Nurmagomedov,MackenzieDernvs.LoopyGodinez,W/L,Women's Strawweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Greg Kleynjans,Hadi Mohamed Ali 28 - 29.Howard Hughes 28 - 29...,...,1555.735723,0.371636,0.349304,0.071634,0.786217,0.193706,0.450410,2.509381,"[tensor(0.7461), tensor(0.8729), tensor(0.5662...",0
292,UFCFightNight:Sandhagenvs.Nurmagomedov,TonyFergusonvs.MichaelChiesa,L/W,Welterweight Bout,Submission,1,3:44,3 Rnd (5-5-5),Marc Goddard,Rear Naked Choke,...,1578.089046,0.349290,0.289054,0.015493,0.910271,0.265372,0.603444,2.240616,"[tensor(0.3938), tensor(0.3355), tensor(0.3099...",0
293,UFCFightNight:Sandhagenvs.Nurmagomedov,TonyFergusonvs.MichaelChiesa,L/W,Welterweight Bout,Submission,1,3:44,3 Rnd (5-5-5),Marc Goddard,Rear Naked Choke,...,1622.714126,0.629883,0.654396,0.206967,0.934329,0.197409,-0.444212,2.459342,"[tensor(0.5987), tensor(0.0508), tensor(0.0537...",1
294,UFCFightNight:Sandhagenvs.Nurmagomedov,MarlonVeravs.DeivesonFigueiredo,L/W,Bantamweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Keith Peterson,Ben Cartlidge 28 - 29.Hadi Mohamed Ali 28 - 29...,...,1684.684440,0.460937,0.431182,0.007864,0.985080,0.331129,0.147688,1.580338,"[tensor(0.5405), tensor(0.2389), tensor(0.3803...",0


In [41]:
import pandas as pd
import numpy as np
import torch


from scipy.stats import bernoulli, norm

def calculate_uncertainty(p_win, num_simulations):
    """
    Calculate uncertainty statistics for a Bernoulli outcome.
    
    Parameters:
        p_win: Probability of the fighter winning.
        num_simulations: Number of simulations run.
    
    Returns:
        variance, std_dev, and Wilson score confidence intervals (lower, upper).
    """
    variance = p_win * (1 - p_win)
    std_dev = np.sqrt(variance)
    
    z = norm.ppf(0.975)
    denominator = 1 + (z**2 / num_simulations)
    center_adjusted_probability = p_win + (z**2 / (2 * num_simulations))
    adjusted_std_dev = np.sqrt((p_win * (1 - p_win) + (z**2 / (4 * num_simulations))) / num_simulations)
    
    lower_bound = (center_adjusted_probability - z * adjusted_std_dev) / denominator
    upper_bound = (center_adjusted_probability + z * adjusted_std_dev) / denominator
    
    return variance, std_dev, lower_bound, upper_bound

def simulate_fights_by_bout_with_uncertainty(df, bout_col, trace_col, fighter_col, num_simulations=10000):
    """
    Simulate future fights for each bout by comparing tensor traces of both fighters,
    and use Bernoulli distribution to model the fight outcome, along with uncertainty statistics.
    
    Parameters:
        df: DataFrame containing fight data.
        bout_col: Column name for BOUT (to group by).
        trace_col: Column name for fighter's trace (tensor).
        fighter_col: Column name for the fighter's name.
        num_simulations: Number of simulations to run.
    
    Returns:
        A DataFrame containing the win probability, Bernoulli outcome, and uncertainty statistics.
    """
    bout_groups = df.groupby(bout_col)
    
    probabilities = []

    for bout, group in bout_groups:
        if len(group) == 2:  
            fighter_A = group.iloc[0] 
            fighter_B = group.iloc[1] 
            
            trace_A = fighter_A[trace_col].numpy()
            trace_B = fighter_B[trace_col].numpy()
            
            wins_A, wins_B = 0, 0
            
            for _ in range(num_simulations):
                sample_A = np.random.choice(trace_A)  
                sample_B = np.random.choice(trace_B)  
                
                if sample_A > sample_B:
                    wins_A += 1
                else:
                    wins_B += 1
            
            prob_A_win = wins_A / num_simulations
            prob_B_win = wins_B / num_simulations
            
            fight_outcome_A = bernoulli.rvs(prob_A_win)  
            fight_outcome_B = 1 - fight_outcome_A  
            
            variance_A, std_dev_A, lower_A, upper_A = calculate_uncertainty(prob_A_win, num_simulations)
            variance_B, std_dev_B, lower_B, upper_B = calculate_uncertainty(prob_B_win, num_simulations)
            
            for index, fighter in group.iterrows():
                if fighter[fighter_col] == fighter_A[fighter_col]:
                    probabilities.append({
                        'index': index,
                        'win_probability': prob_A_win,
                        'bernoulli_outcome': fight_outcome_A,  
                        'variance': variance_A,
                        'std_dev': std_dev_A,
                        'lower_CI': lower_A,
                        'upper_CI': upper_A
                    })
                elif fighter[fighter_col] == fighter_B[fighter_col]:
                    probabilities.append({
                        'index': index,
                        'win_probability': prob_B_win,
                        'bernoulli_outcome': fight_outcome_B, 
                        'variance': variance_B,
                        'std_dev': std_dev_B,
                        'lower_CI': lower_B,
                        'upper_CI': upper_B
                    })
    
    prob_df = pd.DataFrame(probabilities).set_index('index')
    
    return prob_df


simulation_results = simulate_fights_by_bout_with_uncertainty(extra_val, 'BOUT', 'trace', 'FIGHTER')
df_merged = extra_val.join(simulation_results)

In [ ]:
from copulas.multivariate import GaussianMultivariate
from copulas.univariate import GaussianKDE
from copulas.multivariate import GaussianMultivariate
from copulas.univariate import GaussianUnivariate

def simulate_fights_with_copulas(df, bout_col, trace_col, fighter_col, num_simulations=10000):
    """
    Simulate future fights using copulas by modeling dependency between fighter performances.
    
    Parameters:
        df: DataFrame containing fight data.
        bout_col: Column name for BOUT (to group by).
        trace_col: Column name for fighter's trace (tensor).
        fighter_col: Column name for the fighter's name.
        num_simulations: Number of simulations to run.
    
    Returns:
        A DataFrame containing the win probability for the fighter in the row.
    """
    bout_groups = df.groupby(bout_col)
    
    probabilities = []
    
    for bout, group in bout_groups:
        if len(group) == 2:  
            fighter_A = group.iloc[0]  
            fighter_B = group.iloc[1]  
            
            trace_A = fighter_A[trace_col].numpy()
            trace_B = fighter_B[trace_col].numpy()
            
            marginal_A = GaussianUnivariate()
            marginal_B = GaussianUnivariate()
            
            try:
                marginal_A.fit(trace_A)
                marginal_B.fit(trace_B)
            except Exception as e:
                print(f"Error fitting marginals for bout {bout}: {e}")
                continue  
            
            joint_data = np.column_stack([trace_A, trace_B])
            
            copula = GaussianMultivariate()
            try:
                copula.fit(joint_data)
            except Exception as e:
                print(f"Error fitting copula for bout {bout}: {e}")
                continue  
            
            simulated_outcomes = copula.sample(num_simulations)
            simulated_outcomes = np.array(simulated_outcomes)
            
            if simulated_outcomes.shape[1] != 2:
                print(f"Unexpected shape for simulated outcomes: {simulated_outcomes.shape}")
                continue  
            
            simulated_A = simulated_outcomes[:, 0]
            simulated_B = simulated_outcomes[:, 1]
            
            wins_A = np.sum(simulated_A > simulated_B)
            prob_A_win = wins_A / num_simulations
            prob_B_win = 1 - prob_A_win
            
            for index, fighter in group.iterrows():
                if fighter[fighter_col] == fighter_A[fighter_col]:
                    probabilities.append({
                        'index': index,
                        'copula_win_probability': prob_A_win
                    })
                elif fighter[fighter_col] == fighter_B[fighter_col]:
                    probabilities.append({
                        'index': index,
                        'copula_win_probability': prob_B_win
                    })
    
    prob_df = pd.DataFrame(probabilities).set_index('index')
    
    return prob_df

probabilities_df = simulate_fights_with_copulas(df_merged, 'BOUT', 'trace', 'FIGHTER')

df_merged = df_merged.join(probabilities_df)


In [42]:
df_merged

,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,...,skew,kurtosis,trace,Estimate,win_probability,bernoulli_outcome,variance,std_dev,lower_CI,upper_CI
0,UFCFightNight:Ankalaevvs.Walker2,MagomedAnkalaevvs.JohnnyWalker,W/L,Light Heavyweight Bout,KO/TKO,2,2:42,5 Rnd (5-5-5-5-5),Marc Goddard,Punch to Head At Distance,...,-0.222904,2.601757,"[tensor(0.7603), tensor(0.5474), tensor(0.9006...",1,0.8452,0,0.130837,0.361714,0.837978,0.852157
1,UFCFightNight:Ankalaevvs.Walker2,JimMillervs.GabrielBenitez,W/L,Lightweight Bout,Submission,3,3:25,3 Rnd (5-5-5),Dan Miragliotta,Neck Crank From Back Control,...,0.127930,2.261112,"[tensor(0.5753), tensor(0.4763), tensor(0.0156...",0,0.6923,1,0.213021,0.461542,0.683182,0.701271
2,UFCFightNight:Ankalaevvs.Walker2,JimMillervs.GabrielBenitez,W/L,Lightweight Bout,Submission,3,3:25,3 Rnd (5-5-5),Dan Miragliotta,Neck Crank From Back Control,...,0.280852,1.755708,"[tensor(0.5394), tensor(0.2533), tensor(0.0196...",0,0.3077,0,0.213021,0.461542,0.298729,0.316818
3,UFCFightNight:Ankalaevvs.Walker2,MagomedAnkalaevvs.JohnnyWalker,W/L,Light Heavyweight Bout,KO/TKO,2,2:42,5 Rnd (5-5-5-5-5),Marc Goddard,Punch to Head At Distance,...,-0.019269,2.198014,"[tensor(0.6817), tensor(0.5073), tensor(0.0386...",1,0.1548,1,0.130837,0.361714,0.147843,0.162022
4,UFCFightNight:Ankalaevvs.Walker2,MatthewSemelsbergervs.PrestonParsons,L/W,Welterweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Dan Miragliotta,Mike Bell 27 - 30.Junichiro Kamijo 27 - 30.Ron...,...,-0.127112,2.495623,"[tensor(0.7042), tensor(0.7541), tensor(0.8722...",1,0.9238,1,0.070394,0.265318,0.918436,0.928839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,UFCFightNight:Sandhagenvs.Nurmagomedov,MackenzieDernvs.LoopyGodinez,W/L,Women's Strawweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Greg Kleynjans,Hadi Mohamed Ali 28 - 29.Howard Hughes 28 - 29...,...,0.450410,2.509381,"[tensor(0.7461), tensor(0.8729), tensor(0.5662...",0,0.4775,1,0.249494,0.499493,0.467721,0.487297
292,UFCFightNight:Sandhagenvs.Nurmagomedov,TonyFergusonvs.MichaelChiesa,L/W,Welterweight Bout,Submission,1,3:44,3 Rnd (5-5-5),Marc Goddard,Rear Naked Choke,...,0.603444,2.240616,"[tensor(0.3938), tensor(0.3355), tensor(0.3099...",0,0.7718,1,0.176125,0.419672,0.763471,0.779920
293,UFCFightNight:Sandhagenvs.Nurmagomedov,TonyFergusonvs.MichaelChiesa,L/W,Welterweight Bout,Submission,1,3:44,3 Rnd (5-5-5),Marc Goddard,Rear Naked Choke,...,-0.444212,2.459342,"[tensor(0.5987), tensor(0.0508), tensor(0.0537...",1,0.2282,0,0.176125,0.419672,0.220080,0.236529
294,UFCFightNight:Sandhagenvs.Nurmagomedov,MarlonVeravs.DeivesonFigueiredo,L/W,Bantamweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Keith Peterson,Ben Cartlidge 28 - 29.Hadi Mohamed Ali 28 - 29...,...,0.147688,1.580338,"[tensor(0.5405), tensor(0.2389), tensor(0.3803...",0,0.6143,1,0.236936,0.486760,0.604718,0.623795


In [43]:
df_merged.to_csv('MergedWJointProba.csv')

In [44]:
print(extra_val)

                                      EVENT  \
0          UFCFightNight:Ankalaevvs.Walker2   
1          UFCFightNight:Ankalaevvs.Walker2   
2          UFCFightNight:Ankalaevvs.Walker2   
3          UFCFightNight:Ankalaevvs.Walker2   
4          UFCFightNight:Ankalaevvs.Walker2   
..                                      ...   
291  UFCFightNight:Sandhagenvs.Nurmagomedov   
292  UFCFightNight:Sandhagenvs.Nurmagomedov   
293  UFCFightNight:Sandhagenvs.Nurmagomedov   
294  UFCFightNight:Sandhagenvs.Nurmagomedov   
295  UFCFightNight:Sandhagenvs.Nurmagomedov   

                                     BOUT OUTCOME               WEIGHTCLASS  \
0          MagomedAnkalaevvs.JohnnyWalker     W/L    Light Heavyweight Bout   
1              JimMillervs.GabrielBenitez     W/L          Lightweight Bout   
2              JimMillervs.GabrielBenitez     W/L          Lightweight Bout   
3          MagomedAnkalaevvs.JohnnyWalker     W/L    Light Heavyweight Bout   
4    MatthewSemelsbergervs.PrestonParson

In [45]:
mean_probabilities.numpy()

array([0.5801081 , 0.4585257 , 0.4323169 , 0.50935143, 0.5472917 ,
       0.5520411 , 0.53577036, 0.46278095, 0.4001665 , 0.52934796,
       0.56189096, 0.5755588 , 0.42246875, 0.465054  , 0.42025167,
       0.42766982, 0.47980595, 0.6170276 , 0.3864632 , 0.51095474,
       0.5058907 , 0.42671216, 0.30419928, 0.48635358, 0.4583503 ,
       0.57465154, 0.6299626 , 0.48876756, 0.41015103, 0.5286673 ,
       0.48439196, 0.35822797, 0.5882188 , 0.49671787, 0.48125112,
       0.3800105 , 0.35521674, 0.5450408 , 0.5331386 , 0.4371745 ,
       0.43014073, 0.47124648, 0.43237993, 0.5139048 , 0.45399052,
       0.63685906, 0.61648184, 0.42708164, 0.5269279 , 0.37439087,
       0.57551146, 0.40920386, 0.6897586 , 0.42058876, 0.4170358 ,
       0.589378  , 0.7711118 , 0.6616321 , 0.41225284, 0.49300328,
       0.42098057, 0.4491744 , 0.4041614 , 0.36213556, 0.46098614,
       0.5295415 , 0.5422469 , 0.5414645 , 0.4995089 , 0.6243174 ,
       0.46727362, 0.3465054 , 0.4045608 , 0.72734153, 0.53033

In [46]:
print(classification_report(y_tensor_val.int(), predicted_labels))

              precision    recall  f1-score   support

           0       0.61      0.70      0.65       148
           1       0.65      0.55      0.60       148

    accuracy                           0.62       296
   macro avg       0.63      0.62      0.62       296
weighted avg       0.63      0.62      0.62       296



In [47]:
class_counts = torch.bincount(torch.tensor(y_train.values, dtype=torch.long))
class_counts

tensor([4510, 4510])

In [48]:
import statsmodels.api as sm
import pandas as pd


feats = ['delta_ELO_FIGHTER', 'delta_AGE', 'delta_REACH', 'delta_SIG_STR_vs_avoid', 'delta_TD_vs_avoid',
         'opponent_PC1', 'opponent_PC2', 'delta_CUM_CTRL_TIME_IN_SECONDS_PM', 'STANCE', 'AGE', 'ELO_FIGHTER']

X = X_train[feats]
X = sm.add_constant(X)

y = y_train

logit_model = sm.Logit(y, X)
result = logit_model.fit()

print(result.summary())

X_test['predicted_prob'] = result.predict(sm.add_constant(X_test[feats]))
X_test['predicted_class'] = (X_test['predicted_prob'] > 0.5).astype(int)

accuracy = accuracy_score(y_test, X_test['predicted_class'])
print(f"Accuracy: {accuracy:.4f}")

Optimization terminated successfully.
         Current function value: 0.663877
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                 RESULT   No. Observations:                 9020
Model:                          Logit   Df Residuals:                     9008
Method:                           MLE   Df Model:                           11
Date:                Tue, 10 Sep 2024   Pseudo R-squ.:                 0.04223
Time:                        18:19:36   Log-Likelihood:                -5988.2
converged:                       True   LL-Null:                       -6252.2
Covariance Type:            nonrobust   LLR p-value:                3.343e-106
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const                                 0.4477      0.636      0

In [49]:
print(classification_report(X_test['predicted_class'], y_test))

              precision    recall  f1-score   support

           0       0.54      0.57      0.56       257
           1       0.60      0.56      0.58       287

    accuracy                           0.57       544
   macro avg       0.57      0.57      0.57       544
weighted avg       0.57      0.57      0.57       544

